| Name | Type (Shape) | Description |
| --- | --- | --- |
| closed_null_geodesics | list (N,) | list containing $ N $ closed null geodesics
| vortex | list | list containing extracted vortices

In [1]:
# Import numpy
import numpy as np

# Import shapely library to create polygons and points
from shapely.geometry import Polygon, Point

# Import math tools
from math import sqrt

In [2]:
def _elliptic_LCS(closed_null_geodesics):
    
    Area = []
    Poly = []
    return_distance = []
    
    for geodesics in closed_null_geodesics:
        
        if np.sum(geodesics[0]) is not None:
            
            x = geodesics[0]
            y = geodesics[1]
        
            # compute Polygon object from boundary of closed null geodesics
            Poly.append(Polygon(zip(x, y)))
            
            # compute area of closed null geodesics
            # Area.append(Poly[-1].area)
            # compute re-intersection distance of closed null geodesics
            return_distance.append(sqrt((x[-1]-x[0])**2+(y[-1]-y[0])**2))

    # if no vortex is found then return 'nan'
    if len(return_distance) == 0:
        return None, None
    
    if len(return_distance) == 1:
        return x, y
    
    # sort geodesic curves from maximum re-intersection distance to lowest re-intersection distance:
    return_distance_sorted, closed_null_geodesics_sorted = zip(*sorted(zip(return_distance, closed_null_geodesics), reverse = True))
    return_distance_sorted, Poly_sorted = zip(*sorted(zip(return_distance, Poly), reverse = True))
    
    closed_elliptic_OECS = []
    
    for idx, Poly_geodesics in enumerate(Poly_sorted):
        
        coords = np.array(Poly_sorted[idx].exterior.coords)
        
        x = coords[:, 0]
        y = coords[:, 1]
        
        # compute centroid of polygon
        x_c = np.mean(x)
        y_c = np.mean(y)
        
        # If first closed elliptic LCS, then set the center as a reference
        if idx == 0:
            
            closed_elliptic_OECS.append([x_c, y_c, Poly_geodesics])
        
        # Set boolean to create new vortex
        Bool_create_new_closed_elliptic_OECS = True
        
        # iterate over all elliptic OECS
        for v in range(len(closed_elliptic_OECS)):
        
            # If vortex center is contained inside the closed null geodesics --> Bool_create_new_vortex = False
            # --> no new vortex is create
            if Poly_geodesics.contains(Point(closed_elliptic_OECS[v][0], closed_elliptic_OECS[v][1])):
                
                closed_elliptic_OECS[v] = [x_c, y_c, Poly_geodesics]
                
                Bool_create_new_closed_elliptic_OECS = False
                
        # If vortex center is not contained inside the closed null geodesics, then a new vortex is create    
        if Bool_create_new_closed_elliptic_OECS:
            
            closed_elliptic_OECS.append([x_c, y_c, Poly_geodesics])        
    
    # Create list storing closed elliptic OECS with minimal return distance
    min_return_elliptic_OECS = []
    
    # Iteratere over all closed elliptic OECS and append them to 'closed_elliptic_OECS'
    for v in closed_elliptic_OECS:
        
        coords = np.array(v[2].exterior.coords)
        
        x = coords[:, 0]
        y = coords[:, 1]
        
        min_return_elliptic_OECS.append([x, y])
    
    x_elliptic, y_elliptic = [], []
    
    # Store elliptic OECSs with minimum return distance in list
    if min_return_elliptic_OECS:
        
        for elliptic in min_return_elliptic_OECS:
            
            x_elliptic.append(elliptic[0])
            y_elliptic.append(elliptic[1])
    
    return x_elliptic, y_elliptic